## Homework-4 ##

### Question - 5 : SDP Problem

Solve the following SDP problem using the cutting plane algorithm in python.

$$
\begin{aligned}
    \max_{X\in \mathcal{S}^{4}} \sum_{i,j,i\neq j} Q_{ij} \cdot (1 - X_{ij})\\
\text{diag}(X) = e\\
X\succeq 0
\end{aligned}
$$

where
$
Q = \begin{bmatrix}
0 & 1 & 3 & 1 \\
1 & 0 & 0 & 2 \\
3 & 0 & 0 & 4 \\
1 &2 & 4 & 0 \\
\end{bmatrix}
$


$\text{diag}(X) = e$ means the diagonal entries of $X$ equal to 1.

In [1]:
# Including libraries
from pyomo.environ import *
import numpy as np
import pprint

In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [3]:
# Creating a Pyomo model
model = ConcreteModel()

# Define sets
model.I = RangeSet(4)

# Define variables
model.X = Var(model.I, model.I)

# Define parameters
def Q_init(model, i, j):
    Q = np.array([[0, 1, 3, 1], [1, 0, 0, 2], [3, 0, 0, 4], [1, 2, 4, 0]])
    return Q[i-1, j-1]

model.Q = Param(model.I, model.I, initialize=Q_init)


In [4]:
# Define objective function

model.obj = Objective(expr= sum(model.Q[(i,j)]*(1- model.X[(i,j)]) for i in model.I for j in model.I if i!=j), sense=maximize)


In [5]:
# Define constraints

def const_rule1(model, i): # Constraint to set the diagonal values of X to be 1
    return model.X[(i,i)] == 1

model.const1 = Constraint(model.I, rule = const_rule1)


def const_rule2(model,  i, j): # Constraint to keep X matrix symmetric
    if i!=j:
        return model.X[(i,j)] == model.X[(j,i)]
    else:
        return Constraint.Skip

model.const4 = Constraint(model.I, model.I, rule = const_rule2)

 Initially, even before we implement the cutting plane algorithm, we start with the linear constraint $(X_{ii} = 1)$ for all $i$ in the range $(1,4)$, along with the symmetry constraint $X_{ij} = X_{ji}$ for all pairs $(i,j)$ where $i,j$ range from $1$ to $3$ and $i \neq j$. Consequently, the problem remains unbounded during this initial phase. This occurs due to the presence of the term $1 - X_{ij}$ in the objective function, where $X_{ij}$ represents the off-diagonal elements. Without bounds on these elements, when these elements are multiplied by a positive quantity $Q_{ij}$, they have the potential to approach $-\infty$, thereby causing the objective value to tend towards $+\infty$, rendering the problem unbounded. To ensure the problem is bounded, we utilize the property of positive semi-definite matrices, which dictates that its principal minors must always be non-negative.

Considering the minor of a principal diagonal element:

\begin{vmatrix}
X_{ii} & X_{ij} \\
X_{ji} & X_{jj} \\
\end{vmatrix}

We require this minor to be non-negative, thus yielding the inequality:

$$X_{ij} * X_{ji} <= X_{ii}*X_{jj}$$

Given the symmetry of matrix X, we can express this inequality as:

$$X_{ij}^2 <= X_{ii}*X_{jj}$$

Since we have set all diagonal elements to be 1, we ultimately obtain:

$$-1 <= X_{ij} <= 1$$





In [6]:
# Constraints to set bounds on off-diagonal elements of X

def const_rule3(model, i, j):
    if i !=j:
        return model.X[(i,j)] <= 1
    else:
        return Constraint.Skip

model.const2 = Constraint(model.I, model.I, rule = const_rule3)

def const_rule4(model, i, j):
    if i !=j:
        return model.X[(i,j)] >= -1
    else:
        return Constraint.Skip

model.const3 = Constraint(model.I, model.I, rule = const_rule4)

In [7]:
# Setting the solver
solver = SolverFactory('glpk')  # Using Gurobi solver

# Creating a constraint list object in Pyomo to iteratively add constraints
model.cuts = ConstraintList()

#  Cutting plane algorithm:

while True:

    results = solver.solve(model)

    # Check for solver termination status
    if results.solver.termination_condition != TerminationCondition.optimal:
        print("Solver failed to converge.")
        break

    # Extract the optimal solution
    X_val = np.array([[model.X[i, j].value for j in model.I] for i in model.I])

    # Perform eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eig(X_val)

    # Find the smallest eigenvalue
    smallest_eigenvalue = min(eigenvalues)
    print("Smallest eigenvalue:", smallest_eigenvalue)

    # Check termination condition
    if smallest_eigenvalue >= -1e-4:
        print("Optimal solution found.")
        break

    # Find the eigen vector corresponding to the smallest eigen value
    v = eigenvectors[:, np.argmin(eigenvalues)]

    # Add the violated constraint
    expr = sum(v[k-1]*model.X[(k,j)]*v[j-1] for k in model.I for j in model.I)
    model.cuts.add(expr >= 0)


# Print optimal solution
print("Optimum objective function value is:", model.obj())
print("\nOptimal solution:")
for i in model.I:
    for j in model.I:
        print(f"X[{i},{j}] = {model.X[i, j].value}")

print("\nEigen values of Optimum solution X are:", eigenvalues)

Smallest eigenvalue: -2.0
Smallest eigenvalue: -1.2360679774997896
Smallest eigenvalue: -0.26789883349049104
Smallest eigenvalue: 0.0
Optimal solution found.
Optimum objective function value is: 40.0

Optimal solution:
X[1,1] = 1.0
X[1,2] = -1.0
X[1,3] = -1.0
X[1,4] = 1.0
X[2,1] = -1.0
X[2,2] = 1.0
X[2,3] = 1.0
X[2,4] = -1.0
X[3,1] = -1.0
X[3,2] = 1.0
X[3,3] = 1.0
X[3,4] = -1.0
X[4,1] = 1.0
X[4,2] = -1.0
X[4,3] = -1.0
X[4,4] = 1.0

Eigen values of Optimum solution X are: [0. 4. 0. 0.]


In [8]:
model.cuts.display()

cuts : Size=3
    Key : Lower : Body                : Upper
      1 :   0.0 :                 0.0 :  None
      2 :   0.0 :   0.552786404500042 :  None
      3 :   0.0 : 0.04559698912211922 :  None
